In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

Selecting previously unselected package libgflags2.2.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

## Detect poses on a test video



In [ ]:
import os
os.chdir('/content')

In [ ]:
!cd openpose && ./build/examples/openpose/openpose.bin --video ../comparativadtw.mp4 --write_json ./output/ --display 0  --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 21.170070 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libs

In [ ]:
! sudo apt install zip unzip

In [ ]:
! zip -r data.zip /content/openpose/output output.mp4

  adding: content/openpose/output/ (stored 0%)
  adding: content/openpose/output/salida_paula_000000000024_keypoints.json (deflated 61%)
  adding: content/openpose/output/salida_paula_000000000029_keypoints.json (deflated 51%)
  adding: content/openpose/output/salida_paula_000000000060_keypoints.json (deflated 51%)
  adding: content/openpose/output/salida_paula_000000000085_keypoints.json (deflated 68%)
  adding: content/openpose/output/salida_paula_000000000001_keypoints.json (deflated 51%)
  adding: content/openpose/output/salida_paula_000000000049_keypoints.json (deflated 52%)
  adding: content/openpose/output/salida_paula_000000000012_keypoints.json (deflated 51%)
  adding: content/openpose/output/salida_paula_000000000075_keypoints.json (deflated 52%)
  adding: content/openpose/output/salida_paula_000000000096_keypoints.json (deflated 56%)
  adding: content/openpose/output/salida_paula_000000000025_keypoints.json (deflated 61%)
  adding: content/openpose/output/salida_paula_000000

In [ ]:
! rm /content/openpose/output/*
! rm data.zip

rm: cannot remove '/content/openpose/output/*': No such file or directory


Validación con coco

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

dataDir='/content/openpose'
dataType='val2017'

# initialize COCO api for person keypoints annotations
annFile = '{}/annotations/person_keypoints_{}.json'.format(dataDir,dataType)
coco_kps=COCO(annFile)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [ ]:
catIds = coco_kps.getCatIds(catNms=['person']);
imgIds = coco_kps.getImgIds(catIds=catIds)
print(len(imgIds))

2693


In [ ]:
num_person_dict = dict()
for imgId in imgIds:
  annIds = coco_kps.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
  anns = coco_kps.loadAnns(annIds)

  num_person = 0
  for an in anns:
    if an['num_keypoints'] > 0:
      num_person = num_person + 1

  if num_person not in num_person_dict:
    num_person_dict[num_person] = []
  
  num_person_dict[num_person].append(imgId)

In [ ]:
picked_images = []
for num_person in num_person_dict:
  img_info = []
  img_info.append(num_person)
  img_info.append(num_person_dict[num_person][0])
  picked_images.append(img_info)

print(picked_images)

[[1, 532481], [4, 458755], [2, 8211], [3, 327701], [0, 188439], [6, 319534], [5, 81988], [13, 122962], [12, 114907], [7, 377113], [10, 57672], [9, 123213], [8, 516677], [11, 172946]]


In [ ]:
for imgPicked in picked_images:
  img = coco_kps.loadImgs(imgPicked[1])
  img = coco_kps.loadImgs(imgPicked[1])[0]
  I = io.imread(img['coco_url'])
  io.imsave('./media/'+str(imgPicked[0])+'.jpg',I)

In [ ]:
os.chdir('/content')
!cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../openpose/annotations/imgsave --write_json ./output/ --display 0  --write_images ../outputMedia

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 11.992116 seconds.


In [ ]:
  annIds = coco_kps.getAnnIds(imgIds=498747, catIds=catIds, iscrowd=None)
  anns = coco_kps.loadAnns(annIds)
  for an in anns:
    print(an['num_keypoints'])

9
10
5
2
6
3
5
5
5
5


# Tiempo de inferéncia

Crear en /content un directorio media y run dentro de media, subir el archivo person_keypoints_val2017.json a /content

In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

dataDir='/content'
dataType='val2017'

# initialize COCO api for person keypoints annotations
annFile = '{}/person_keypoints_{}.json'.format(dataDir,dataType)
coco_kps=COCO(annFile)

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


In [ ]:
#Load all imgIds in category person
catIds = coco_kps.getCatIds(catNms=['person']);
imgIds = coco_kps.getImgIds(catIds=catIds)

In [ ]:
#Create dictionary by num of person detected in image by COCO annotation 
num_person_dict = dict()
for imgId in imgIds:
  annIds = coco_kps.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
  anns = coco_kps.loadAnns(annIds)

  num_person = 0
  for an in anns:
    if an['num_keypoints'] > 0:
      num_person = num_person + 1

  if num_person not in num_person_dict:
    num_person_dict[num_person] = []
  
  num_person_dict[num_person].append(imgId)

In [ ]:
#Picked first image of every category
picked_images = []
for num_person in num_person_dict:
  img_info = []
  img_info.append(num_person)
  img_info.append(num_person_dict[num_person][0])
  picked_images.append(img_info)

print(picked_images)

[[1, 532481], [4, 458755], [2, 8211], [3, 327701], [0, 188439], [6, 319534], [5, 81988], [13, 122962], [12, 114907], [7, 377113], [10, 57672], [9, 123213], [8, 516677], [11, 172946]]


In [ ]:
for imgPicked in picked_images:
  if imgPicked[0] == 5 or imgPicked[0] == 10 or imgPicked[0] == 1:
    img = coco_kps.loadImgs(imgPicked[1])
    img = coco_kps.loadImgs(imgPicked[1])[0]
    I = io.imread(img['coco_url'])
    io.imsave('./media/'+str(imgPicked[0])+'.jpg',I)

In [ ]:
import cv2
import numpy as np
import time

createImgArray = [[15,5,10],[20,10,10],[30,15,15],[40,20,20],[60,30,30]]
for imgArray in createImgArray:
  img1 = cv2.imread('./media/'+str(imgArray[1])+'.jpg')
  img2 = cv2.imread('./media/'+str(imgArray[2])+'.jpg')
  vis = np.concatenate((img1, img2), axis=0)
  cv2.imwrite('./media/'+str(imgArray[0])+'.jpg', vis)

In [ ]:
os.chdir('/content')
imgTime = [[1],[5],[10],[15],[20],[30],[40],[60]]
for imgArray in imgTime:
  nameFile = str(imgArray[0])+'.jpg'
  !mv /content/media/"$nameFile" /content/media/run/"$nameFile"
  start_time = time.time()
  !cd openpose && ./build/examples/openpose/openpose.bin --image_dir ../media/run --write_json ../output/ --display 0  --write_images ../outputMedia
  print("--- %s seconds ---" % (time.time() - start_time))
  !mv /content/media/run/"$nameFile" /content/media/"$nameFile"


Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 8.264517 seconds.
--- 10.649041891098022 seconds ---
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 2.208085 seconds.
--- 2.7275965213775635 seconds ---
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 2.115083 seconds.
--- 2.722130298614502 seconds ---
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total 